<a href="https://colab.research.google.com/github/vincetofy223/federated-privacy-ml-paper/blob/main/federated_learning_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y tensorflow keras flwr numpy
!pip install --upgrade pip


Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Successfully uninstalled tensorflow-2.17.1
Found existing installation: keras 3.11.3
Uninstalling keras-3.11.3:
  Successfully uninstalled keras-3.11.3
Found existing installation: flwr 1.9.0
Uninstalling flwr-1.9.0:
  Successfully uninstalled flwr-1.9.0
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


In [2]:
!pip install tensorflow==2.17.1 flwr==1.9.0 numpy==1.26.4


  Using cached tensorflow-2.17.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached flwr-1.9.0-py3-none-any.whl.metadata (15 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached keras-3.11.3-py3-none-any.whl.metadata (5.9 kB)
Using cached tensorflow-2.17.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (601.4 MB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
Using cached flwr-1.9.0-py3-none-any.whl (364 kB)
Using cached keras-3.11.3-py3-none-any.whl (1.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [flwr]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.17.1 which is incompatible.
thinc 8.3.6 requires nu

In [1]:
import tensorflow as tf
import flwr as fl
import numpy as np

print("TensorFlow version:", tf.__version__)
print("Flower version:", fl.__version__)
print("NumPy version:", np.__version__)


TensorFlow version: 2.17.1
Flower version: 1.9.0
NumPy version: 1.26.4


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [2]:
import flwr as fl
import tensorflow as tf
from tensorflow import keras
import numpy as np

print("✅ Libraries imported successfully")

# Load and preprocess MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = (x_train / 255.0).astype("float32")
x_test = (x_test / 255.0).astype("float32")

# Flatten images from 28x28 -> 784
x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

print("✅ MNIST data loaded successfully")
print(f"Training samples: {x_train.shape[0]}, Test samples: {x_test.shape[0]}")


✅ Libraries imported successfully
✅ MNIST data loaded successfully
Training samples: 60000, Test samples: 10000


In [3]:
# Number of federated clients
NUM_CLIENTS = 5

# Split training data into multiple clients
client_data = []
client_size = len(x_train) // NUM_CLIENTS

for i in range(NUM_CLIENTS):
    start = i * client_size
    end = (i + 1) * client_size
    client_data.append((x_train[start:end], y_train[start:end]))

print(f"✅ Data split into {NUM_CLIENTS} clients successfully")

# Define a simple neural network model
def create_model():
    model = keras.Sequential([
        keras.layers.Input(shape=(784,)),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Test model creation
model = create_model()
print("✅ Model created and compiled successfully")
model.summary()


✅ Data split into 5 clients successfully
✅ Model created and compiled successfully


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
import flwr as fl
import numpy as np
import tensorflow as tf

# Define the custom Flower client for MNIST
class MnistClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_test, y_test):
        self.model = model
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=1, batch_size=32, verbose=0)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, verbose=0)
        return float(loss), len(self.x_test), {"accuracy": float(accuracy)}


In [11]:
# ✅ Flower server strategy (fixed)
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=5,
    min_evaluate_clients=5,
    min_available_clients=5,
    evaluate_metrics_aggregation_fn=lambda metrics: {
        "accuracy": np.mean([m[1]["accuracy"] for m in metrics]),
    },
)

# ✅ Start simulation
fl.simulation.start_simulation(
    client_fn=lambda cid: MnistClient(
        model=create_model(),
        x_train=client_data[int(cid)][0],
        y_train=client_data[int(cid)][1],
        x_test=x_test,
        y_test=y_test,
    ),
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy,
)

print("✅ Federated learning simulation completed successfully")



INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
2025-10-22 02:56:53,122	INFO worker.py:2013 -- Started a local Ray instance.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 9279024743.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'object_store_memory': 3976724889.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEn

✅ Federated learning simulation completed successfully


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [13]:
# ============================================
# ✅ Cell 7: Clean + Visualize Training Progress
# ============================================
import matplotlib.pyplot as plt
import numpy as np
import flwr as fl

# --- Cleaned version of your client ---
class MnistClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_test, y_test):
        self.model = model
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=1, batch_size=32, verbose=0)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, verbose=0)
        return float(loss), len(self.x_test), {"accuracy": float(accuracy)}

# ✅ Fix for Flower warning — convert to Client explicitly
def client_fn(cid):
    client = MnistClient(
        model=create_model(),
        x_train=client_data[int(cid)][0],
        y_train=client_data[int(cid)][1],
        x_test=x_test,
        y_test=y_test,
    )
    return fl.client.Client.from_numpy_client(client)


# --- Visualization of training history ---
# The history was automatically printed after simulation
# Let's re-run the simulation briefly to store the results
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=5,
    min_evaluate_clients=5,
    min_available_clients=5,
)

history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy,
)

# --- Plot training metrics ---
losses = [x[1] for x in history.losses_distributed]
accuracies = [x[1] for x in history.metrics_distributed["accuracy"]]

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(losses)+1), losses, marker='o')
plt.title("Global Model Loss per Round")
plt.xlabel("Round")
plt.ylabel("Loss")

plt.subplot(1, 2, 2)
plt.plot(range(1, len(accuracies)+1), accuracies, marker='o', color='green')
plt.title("Global Model Accuracy per Round")
plt.xlabel("Round")
plt.ylabel("Accuracy")

plt.tight_layout()
plt.show()

print("✅ Visualization complete — clean and professional output.")


INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
2025-10-22 03:08:00,237	INFO worker.py:2013 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3976265318.0, 'node:172.28.0.12': 1.0, 'memory': 9277952410.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEn

RuntimeError: Simulation crashed.

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [6]:
!pip install -U "flwr[simulation]==1.9.0" ray==2.50.1
